In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('seeds_dataset.csv')
df.head()

,ID,area,perimeter,compactness,lengthOfKernel,widthOfKernel,asymmetryCoefficient,lengthOfKernelGroove,seedType
0,1,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,2,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,3,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,4,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,5,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [3]:
df.shape

(210, 9)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 9 columns):
ID                      210 non-null int64
area                    210 non-null float64
perimeter               210 non-null float64
compactness             210 non-null float64
lengthOfKernel          210 non-null float64
widthOfKernel           210 non-null float64
asymmetryCoefficient    210 non-null float64
lengthOfKernelGroove    210 non-null float64
seedType                210 non-null int64
dtypes: float64(7), int64(2)
memory usage: 14.9 KB


In [5]:
df.isnull().sum()

ID                      0
area                    0
perimeter               0
compactness             0
lengthOfKernel          0
widthOfKernel           0
asymmetryCoefficient    0
lengthOfKernelGroove    0
seedType                0
dtype: int64

In [6]:
df['seedType'].value_counts()

3    70
2    70
1    70
Name: seedType, dtype: int64

In [7]:
df1 = df.sample(frac = 1, random_state= 3)

In [8]:
df2= df1.drop('ID', axis = 1)

In [9]:
df2.head()

,area,perimeter,compactness,lengthOfKernel,widthOfKernel,asymmetryCoefficient,lengthOfKernelGroove,seedType
164,11.14,12.79,0.8558,5.011,2.794,6.388,5.049,3
86,18.88,16.26,0.8969,6.084,3.764,1.649,6.109,2
122,16.17,15.38,0.8588,5.762,3.387,4.286,5.703,2
168,11.35,13.12,0.8291,5.176,2.668,4.337,5.132,3
128,20.16,17.03,0.8735,6.513,3.773,1.910,6.185,2


In [10]:
X= df2.drop('seedType', axis = 1)
y= df2['seedType']

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.3, random_state=3)

In [13]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import lightgbm as lgb

In [14]:
lgbc = lgb.LGBMClassifier()

lgbc = lgb.LGBMClassifier(random_state=3)

params = {'n_estimators': randint(5,100),

'max_depth': randint(2, 20),

'min_child_samples': randint(1, 20),

'num_leaves': randint(5,50)}

rand_search_lgbc = RandomizedSearchCV(lgbc,
param_distributions=params, cv=3, random_state=3, n_jobs=-1)

rand_search_lgbc.fit(X_train, y_train)

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=LGBMClassifier(boosting_type='gbdt',
                                            class_weight=None,
                                            colsample_bytree=1.0,
                                            importance_type='split',
                                            learning_rate=0.1, max_depth=-1,
                                            min_child_samples=20,
                                            min_child_weight=0.001,
                                            min_split_gain=0.0,
                                            n_estimators=100, n_jobs=-1,
                                            num_leaves=31, objective=None,
                                            random_state=3, reg_alpha=0.0,
                                            reg_lambda=0.0, silent=...
                                        'min_child_samples': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000007

In [15]:
print(rand_search_lgbc.best_params_)

{'max_depth': 2, 'min_child_samples': 11, 'n_estimators': 46, 'num_leaves': 15}


In [16]:
predictions = rand_search_lgbc.predict(X_test)

In [17]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [18]:
confusion_matrix(y_test,predictions)

array([[18,  3,  2],
       [ 0, 20,  0],
       [ 1,  0, 19]], dtype=int64)

In [19]:
accuracy_score(y_test,predictions)

0.9047619047619048

In [20]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           1       0.95      0.78      0.86        23
           2       0.87      1.00      0.93        20
           3       0.90      0.95      0.93        20

    accuracy                           0.90        63
   macro avg       0.91      0.91      0.90        63
weighted avg       0.91      0.90      0.90        63

